Лабораторная работа №6

Тема: Численное интегрирование

Цель работы: Изучить основные численные формулы интегрирования, особенности их
алгоритмизации.

Вариант 10

$f(x) = \frac{e^{0.2x}tanh(1.5x)}{1.2 + cos(x)}$

1) Найти точное и приближенные значения определенного интеграла из индивидуального
задания в интервале [a=0, b=10], использовав формулы численного интегрирования первого,
второго и четвертого порядка аппроксимации. Подобрать оптимальный шаг h
экспериментально. Для каждого полученного значения определить абсолютную,
относительную погрешности (сопоставив точное и приближенные значения), проверить
теоретическую оценку погрешности ε.

In [ ]:
# подключим пакет интегрирования из библиотеки scipy
from scipy import integrate
import math

# опишем подынтегральную функцию
def target_function_f(x):
 return (math.exp(0.2 * x) * math.tanh(1.5 * x)) / (1.2 + math.cos(x))
# выполним интегрирование
result = integrate.quad(target_function_f, 0.0, 10.0)
# посмотрим результат
print("Точное значение: ", result[0])

Точное значение:  60.797393564866084


Для методов левых и правых прямоугольников:

$\epsilon \leq \frac{1}{2} \cdot 35.837 \cdot (10 - 0) \cdot 0.01 = 1.79185$

Для метода средних прямоугольников:

$\epsilon \leq \frac{1}{24} \cdot 164.289 \cdot (10 - 0) \cdot 0.0001 = 0.006845$

Для метода трапеций:

$\epsilon \leq \frac{1}{12} \cdot 164.289 \cdot (10 - 0) \cdot 0.0001 = 0.013691$

Для метода Симпсона:

$\epsilon \leq \frac{10 - 0}{180} \cdot 5097.638 \cdot 0.00000001 = 0.00000283202$

2) Подготовить программы на языке Rust для всех численных расчетов.

In [ ]:
# Установка Rust
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"
# Проверка установки Rust
!rustc --version

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2024-05-02, rust version 1.78.0 (9b00956e5 2024-04-29)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
  8.0 MiB /   8.0 MiB (100 %)   6.2 MiB/s in  1s ETA:  0s
info: installing component 'clippy'
info: installing component 'rust-docs'
 15.1 MiB /  15.1 MiB (100 %)   1.4 MiB/s in 11s ETA:  0s
info: installing component 'rust-std'
 24.3 MiB /  24.3 MiB (100 %)   6.8 MiB/s in  3s ETA:  0s
info: installing component 'rustc'
 63.7 MiB /  63.7 MiB (100 %)   7.4 MiB/s in  8s ETA:  0s
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu

In [ ]:
%%writefile lab6.rs

fn f(x: f64) -> f64 {
    (f64::exp(0.2 * x) * f64::tanh(1.5 * x)) / (1.2 + f64::cos(x))
}

fn first_l(mut a: f64, b: f64) -> f64 {
    let h = 0.01;
    let n = ((b - a) / h) as i64;
    let mut res = 0.0;

    for _i in 0..n {
        res += f(a) * h;

        a += h;
    };

    res
}

fn first_r(mut a: f64, b: f64) -> f64 {
    let h = 0.01;
    let n = ((b - a) / h) as i64;
    a += h;
    let mut res = 0.0;

    for _i in 0..n {
        res += f(a) * h;

        a += h;
    };

    res
}

fn second_mid_sq(a: f64, b: f64) -> f64 {
    let h = 0.01;
    let n = ((b - a) / h) as i64;
    let mut res = 0.0;
    let mut xim1 = a;
    let mut xi = a + h;

    for _i in 1..n {
        res += f((xim1 + xi) / 2.) * h;

        xim1 += h;
        xi += h;
    };

    res
}

fn second_trap(a: f64, b: f64) -> f64 {
    let h = 0.01;
    let n = ((b - a) / h) as i64;
    let mut res = 0.0;
    let mut xim1 = a;
    let mut xi = a + h;

    for _i in 1..n {
        res += (f(xim1) + f(xi)) * h;

        xim1 += h;
        xi += h;
    };

    res / 2.
}

fn fourth_simp(a: f64, b: f64) -> f64 {
    let h = 0.01;
    let n = ((b - a) / h) as i64;
    let mut res = f(a) + f(b);

    for i in 1..n {
        if i % 2 != 0 {
            res += 4. * f(a + i as f64 * h);
        } else {
            res += 2. * f(a + i as f64 * h);
        }
    };

    res * h / 3.
}

fn main() {
    let a = 0.0;
    let b = 10.0;

    let actual = 60.797393564866084;

    let ans_first_l = first_l(a, b);
    println!("Метод левых прямоугольников: {}", ans_first_l);
    println!("Абсолютная погрешность: {}", actual - ans_first_l);
    println!("Относительная погрешность: {}\n\n", (actual - ans_first_l) / actual);

    let ans_first_r = first_r(a, b);
    println!("Метод правых прямоугольников: {}", ans_first_r);
    println!("Абсолютная погрешность: {}", actual - ans_first_r);
    println!("Относительная погрешность: {}\n\n", (actual - ans_first_r) / actual);

    let ans_second_mid_sq = second_mid_sq(a, b);
    println!("Метод средних прямоугольников: {}", ans_second_mid_sq);
    println!("Абсолютная погрешность: {}", actual - ans_second_mid_sq);
    println!("Относительная погрешность: {}\n\n", (actual - ans_second_mid_sq) / actual);

    let ans_second_trap = second_trap(a, b);
    println!("Метод трапеций: {}", ans_second_trap);
    println!("Абсолютная погрешность: {}", actual - ans_second_trap);
    println!("Относительная погрешность: {}\n\n", (actual - ans_second_trap) / actual);

    let ans_fourth_simp = fourth_simp(a, b);
    println!("Метод Симпсона: {}", ans_fourth_simp);
    println!("Абсолютная погрешность: {}", actual - ans_fourth_simp);
    println!("Относительная погрешность: {}\n\n", (actual - ans_fourth_simp) / actual);
}

Writing lab6.rs


In [ ]:
# Компиляция
!rustc lab6.rs
# Запуск скомпилированной программы
!./lab6

Метод левых прямоугольников: 60.694803076548936
Абсолютная погрешность: 0.10259048831714779
Относительная погрешность: 0.0016874158956780231


Метод правых прямоугольников: 60.89952663520929
Абсолютная погрешность: -0.10213307034320707
Относительная погрешность: -0.001679892251207102


Метод средних прямоугольников: 60.59144197913307
Абсолютная погрешность: 0.2059515857330112
Относительная погрешность: 0.0033875068264773043


Метод трапеций: 60.59109471922997
Абсолютная погрешность: 0.20629884563611256
Относительная погрешность: 0.0033932185828987513


Метод Симпсона: 60.79739356855212
Абсолютная погрешность: -0.0000000036860328123111685
Относительная погрешность: -0.00000000006062813874378444




Вывод: в ходе лабораторной работы изучили основные численные формулы интегрирования, особенности их
алгоритмизации. Метод Симпсона оказался наиболее точным методом интегрирования.